In [1]:
import gym
import ray
from ray.rllib import agents
from copy import deepcopy
import time
ray.init(ignore_reinit_error=True)

2020-06-05 07:56:11,154	INFO resource_spec.py:212 -- Starting Ray with 3.91 GiB memory available for workers and up to 1.96 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-05 07:56:11,604	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.0.11',
 'raylet_ip_address': '192.168.0.11',
 'redis_address': '192.168.0.11:27682',
 'object_store_address': '/tmp/ray/session_2020-06-05_07-56-11_151089_25970/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-05_07-56-11_151089_25970/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-06-05_07-56-11_151089_25970'}

In [2]:
def train_agent(env_name, algo='a3c', iters=50, config_dict={}):
    if hasattr(agents, algo):
        agent = getattr(agents, algo)
#         config = set_config(agent.DEFAULT_CONFIG, config_dict)
        config = agent.DEFAULT_CONFIG
        trainer = getattr(agent, algo.upper() + 'Trainer')(config, 
            env=env_name)
    else:
        raise AttributeError('No attribute {}'.format(algo))
    t0 = time.time()
    results = []
    for n in range(iters):
        t1 = time.time()
        result = trainer.train()
        t2 = time.time()
        print(result['info']['learner'])
        results.append(result)
        if (n + 1) % 10 == 0:
            print("Iter:\t{}\tMean Rewards:\t{:.1f}".format(n+1, result['episode_reward_mean']) + 
                  "\tEps per second:\t{:.3f}\tTotal Time (s):\t{:.1f}".format(
                      result['episodes_this_iter']/(t2-t1), t2-t0))
    
    return trainer, results

In [3]:
trainer, results = train_agent('CartPole-v0')

2020-06-05 07:56:11,946	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-06-05 07:56:12,126	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-06-05 07:59:31,726	WARNING trainer_template.py:124 -- The experimental distributed execution API is enabled for this algorithm. Disable this by setting 'use_exec_api': False.
2020-06-05 07:59:31,837	INFO trainable.py:180 -- _setup took 199.716 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2020-06-05 07:59:31,838	INFO trainable.py:217 -- Getting current IP.
2020-06-05 07:59:31,842	WARNING util.py:37 -- Install gputil for GPU system monitoring.


NotImplementedError: Got <class 'list'>, but expected numpy array or torch tensor.